### Columbia River Basin Data from 2000-2010

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Create output folder
try:
    os.mkdir('./output')
except:
    pass

# Checks for files in data folder
files = os.listdir('./data')
output = os.listdir('./output')


# Strips header from NRCS data, outputs the new CSVs into the output folder
for file_name in files:
    with open('./data/' + str(file_name), 'r') as file_in:
        data = file_in.read().splitlines(True)
    with open('./output/' + str(file_name), 'w') as file_out:
        file_out.writelines(data[77:])

In [3]:
df_list = []
for i in os.listdir('./output'):
    path = os.path.join("./output", i)
    temp = pd.read_csv(path, index_col=None, header=0) 
    df_list.append(temp)

In [4]:
df = pd.concat(df_list, axis=0, ignore_index=True)

In [5]:
df.isnull().sum()

Lat                        264
Long                       264
YYYYMMDD                   264
Basin_name                 264
Station_id                   0
Acton_id                   264
Station_name               264
Elevation                    0
Wteq_amt                   264
Wteq_med                   264
Wteq_amt_pct_med           165
Wteq_amt_pct_med_flag     3168
Prec_wytd_amt              264
Prec_wytd_avg              264
Prec_wytd_pctavg            55
Prec_wytd_pct_avg_flag    3278
                          3080
dtype: int64

In [6]:
df = df[df['YYYYMMDD'].notnull()]

In [7]:
df.isnull().sum()

Lat                          0
Long                         0
YYYYMMDD                     0
Basin_name                   0
Station_id                   0
Acton_id                     0
Station_name                 0
Elevation                    0
Wteq_amt                     0
Wteq_med                     0
Wteq_amt_pct_med           165
Wteq_amt_pct_med_flag     2904
Prec_wytd_amt                0
Prec_wytd_avg                0
Prec_wytd_pctavg            55
Prec_wytd_pct_avg_flag    3014
                          3069
dtype: int64

In [8]:
df['YYYYMMDD'].map(lambda x: int(x))

0       20090210
1       20090210
2       20090210
3       20090210
4       20090210
          ...   
3327    20100210
3328    20100210
3329    20100210
3330    20100210
3331    20100210
Name: YYYYMMDD, Length: 3069, dtype: int64

In [9]:
df['Wteq_amt'].value_counts()

-999.0    47
 0.0      42
 11.6     31
 9.3      30
 8.5      30
          ..
 61.5      1
 28.3      1
 40.3      1
 33.6      1
 45.2      1
Name: Wteq_amt, Length: 442, dtype: int64

In [10]:
df['YYYYMMDD'] = pd.to_datetime(df['YYYYMMDD'], format='%Y%m%d')

In [11]:
df = df.drop(['Wteq_amt_pct_med_flag', 'Prec_wytd_pct_avg_flag', ' '], axis=1)

In [12]:
df = df.replace(-998.0, np.nan)

In [13]:
df = df.replace(-999.0, np.nan)

In [14]:
df[df['Wteq_amt'].isnull()].groupby('Station_id').mean()

,Lat,Long,Wteq_amt,Wteq_med,Wteq_amt_pct_med,Prec_wytd_amt,Prec_wytd_avg,Prec_wytd_pctavg
Station_id,,,,,,,,
1068,47.166667,-121.416667,NaN,NaN,NaN,NaN,NaN,NaN
1084,45.250000,-123.283333,NaN,NaN,NaN,NaN,NaN,NaN
1144,47.983333,-114.350000,NaN,NaN,NaN,NaN,NaN,NaN
1165,48.200000,-114.516667,NaN,NaN,NaN,22.200000,NaN,NaN
549,41.516667,-116.016667,NaN,NaN,NaN,16.781818,17.3,97.0


In [15]:
df = df.dropna(thresh=12)

In [16]:
df.isnull().sum()

Lat                   0
Long                  0
YYYYMMDD              0
Basin_name            0
Station_id            0
Acton_id              0
Station_name          0
Elevation             0
Wteq_amt              0
Wteq_med             88
Wteq_amt_pct_med    110
Prec_wytd_amt         0
Prec_wytd_avg        11
Prec_wytd_pctavg     11
dtype: int64

In [17]:
#list of all my features with nulls after dropping those two flag columns and changing all the codes to nulls as well
features_to_fill = ['Wteq_amt', 'Wteq_med', 'Wteq_amt_pct_med', 'Prec_wytd_avg', 'Prec_wytd_pctavg']
#groups by year, and then fills the nulls with the year's median value
def null_filler(feature):
    return df.groupby(by = df['YYYYMMDD'])[feature].transform(lambda x: x.fillna(x.median()))

In [18]:
#fills the nulls of every feature in my above list
for feature in features_to_fill:
    df[feature] = null_filler(feature)

In [19]:
df.isnull().sum()

Lat                 0
Long                0
YYYYMMDD            0
Basin_name          0
Station_id          0
Acton_id            0
Station_name        0
Elevation           0
Wteq_amt            0
Wteq_med            0
Wteq_amt_pct_med    0
Prec_wytd_amt       0
Prec_wytd_avg       0
Prec_wytd_pctavg    0
dtype: int64

In [20]:
df.columns = df.columns.str.lower()

In [21]:
dummies = pd.get_dummies(df['basin_name'])

In [22]:
df = pd.concat([df, dummies], axis=1)

In [23]:
df = df.drop('basin_name', axis=1)

In [24]:
df.set_index('yyyymmdd', inplace = True)

In [25]:
df.to_csv('./basin_2000_2010.csv')